<a href="https://colab.research.google.com/github/PatilAmartyaJ/NewsMiner/blob/main/News_Miner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium

In [2]:
def generateDaywiseUrl(location,date_obj,pageDictionary,article_array):
    for key in pageDictionary.keys():
     for j in range(1,page_dictionary[key]+1):
      url_prefix = 'https://epaper.lokmat.com/'
      article_url=url_prefix+"articlepage.php?articleid=LOK_"
      article_url+=location_dict[location]+"_"
      article_url+=  date_obj.strftime("%Y%m%d")
      article_url+="_"+str(key)+"_"+str(j)
      article_array.append(article_url)
    return article_array

In [3]:
from bs4 import BeautifulSoup
import requests

def gatherImageUrls(source_url):
  response = requests.get(source_url)
# Parse the HTML
  soup = BeautifulSoup(response.content, 'html.parser')

# Find all <img> tags
  img = soup.find(id='ImageArticle')

# Print image sources
  src = img.get('src')
  parts=source_url.split("_")
  joined= "_".join(parts[1:])
  article_map[joined]=src


In [ ]:
# -*- coding: utf-8 -*-
"""Newspaper_Scrapping.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1zxAVIueLcZzd2DA2SAya1uxAKBokT2r1
"""

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import WebDriverException, TimeoutException
from bs4 import BeautifulSoup
import requests
import os
from datetime import datetime
import time

page_dictionary={}
article_map={}
# Initialize a single driver instance for the entire session
def init_driver():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(options=options)
    return driver

def getUrl(location, date_obj):
   url = 'https://epaper.lokmat.com/main-editions/' + location
   url += "%20Main/"
   url += str(date_obj)
   url += "/"
   url += "1"
   print(url)
   return url

def getArticles(driver, url):
    try:
        driver.get(url)

        # Wait until the element with debug-id="slide-0" is present
        wait = WebDriverWait(driver, 30)
        time.sleep(2)

        # Find all slide containers (each page might have multiple slides)
        slide_containers = driver.find_elements(By.CSS_SELECTOR, '[debug-id^="slide-"]')
        print(f"Found {len(slide_containers)} slide containers")

        total_articles = 0
        for container in slide_containers:
            # Find article links within each container
            links = container.find_elements(By.TAG_NAME, 'a')
            debug_id=container.get_attribute('debug-id')
            part = debug_id.split('-')[1]
            if part.isdigit():
               page_num = int(part) + 1
               page_dictionary[page_num]= len(links)

            total_articles += len(links)

        return total_articles


    except (TimeoutException, WebDriverException) as e:
        print(f"Error getting articles from {url}: {e}")
        return 0

def getPageCount(driver, url):
    try:
        driver.get(url)

        # Wait for the <ul> element to have content
        wait = WebDriverWait(driver, 30)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "ul#mTS_1_container li")))

        # Extract inner HTML
        container = driver.find_element(By.ID, "mTS_1_container")
        html_content = container.get_attribute("innerHTML")

        soup = BeautifulSoup(html_content, 'html.parser')

        # Find all <li> elements
        li_elements = soup.find_all('li')
        print(f"Number of pages: {len(li_elements)}")
        return len(li_elements)

    except (TimeoutException, WebDriverException) as e:
        print(f"Error getting page count from {url}: {e}")
        return 0

def kill_chrome_processes():
    try:
        # Kill any existing Chrome processes
        os.system("pkill -f chrome")
        os.system("pkill -f chromedriver")
    except:
        pass

# Main execution
if __name__ == "__main__":
    # Call this before starting your scraping
    kill_chrome_processes()

    location_dict = {
        "Kolhapur": "KOLK",
        "Mumbai": "MULK",
        "Pune": "PUNK",
        "Nagpur": "NPLK",
        "Nashik": "NSLK",
        "Chhatrapati Sambhajinagar": "AULK",
        "Jalgoan": "JLLK",
        "Akola": "AKLK",
        "Solapur": "SOLK",
        "Ahilyanagar": "ANLK",
        "Goa": "GALK",
        "New Delhi": "DLLK"
    }

    print("Valid locations are: ")
    for key in location_dict.keys():
        print(key)

    location = input("Enter the Location: ")

    date_str = input("Enter a date (YYYY-MM-DD): ")
    date_obj = datetime.strptime(date_str, "%Y-%m-%d").date()

    print("You entered:", date_obj)

    invalid_dates = ["2025-01-27", "2025-03-14", "2025-05-02", "2025-08-16"]
    url_prefix = 'https://epaper.lokmat.com/'

    if date_obj.strftime("%Y-%m-%d") not in invalid_dates and date_obj <= datetime.now().date():
        # Initialize the driver once
        driver = init_driver()

        try:
            url = getUrl(location, date_obj)
            number_of_pages = getPageCount(driver, url)

            print(f"Processing page 1: {url}")
            num_articles = getArticles(driver, url)
            print(f"Number of articles for page number: {num_articles}")
            print(page_dictionary)
            article_list=[]
            article_list=generateDaywiseUrl(location,date_obj,page_dictionary,article_list)
            for article in article_list:
              gatherImageUrls(article)
            print(article_map)
                # for j in range(1, num_articles + 1):
                #     article_url = url_prefix + "articlepage.php?articleid=LOK_"
                #     article_url += location_dict[location] + "_"
                #     article_url += date_obj.strftime("%Y%m%d")
                #     article_url += "_" + str(i) + "_" + str(j)
                #     print(article_url)

                # # Add a small delay to avoid overwhelming the server
                # time.sleep(1)

        finally:
            # Quit the driver only at the end
            driver.quit()

In [ ]:
for key in article_map.keys():
  filename = os.path.basename(article_map[key])
  response = requests.get(article_map[key])
  if response.status_code == 200:
    with open(filename, 'wb') as f:
        f.write(response.content)
    print(f"Image saved as: {filename}")
  else:
    print(f"Failed to download image. Status code: {response.status_code}")